# [모듈 4.1] 평가 및 조건 단계 개발 (SageMaker Model Building Pipeline 평가 및 조건 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 모델 평가 개요 
- 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 2. 모델 평가 코드 확인
- 3. 모델 평가 스텝 개발 및 실행
    - 아래의 3단계를 진행하여 SageMaker Model Building Pipeline 에서 훈련 스텝 개발 함. 아래의 (1), (2) 단계는 옵션이지만, 실제 현업 개발시에 필요한 단계이기에 실행을 권장 드립니다.
        - (1) **[로컬 노트북 인스턴스]**에서 [다커 컨테이너 없이] 스크립트로 
        - (2) **[로컬 노트북 인스턴스]**에서 다커 컨테이너로 훈련 코드 실행 (로컬 모드로 불리움)
        - (3) SageMaker Model Building Pipeline 에서 모델 평가 및 조건 스텝 개발 및 실행
    
---

# 0. 모델 평가 개요
추후 기술 예정 입니다.

# 1. 데이터 세트 로딩 및 기본 훈련 변수 설정
- 이전 단계(전처리)에서 결과 파일을 로딩 합니다. 실제 훈련에 제공되는 데이터를 확인하기 위함 입니다.
---

In [7]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
# role = 'arn:aws:iam::057716757052:role/service-role/AmazonSageMaker-ExecutionRole-20201219T152596'

%store -r 
# %store

### 모델 아피텍트 위치 확인

In [8]:
print("train_model_artifact: \n", train_model_artifact)

train_model_artifact: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/training_jobs/pipelines-fi8k402mf07h-FraudScratchTrain-fnnMKXDW3b/output/model.tar.gz


# 2. 모델 평가 코드 확인


---

평가 스크립트는 `xgboost`를 사용하고 다음을 실행합니다.

* 모델을 로드합니다. 
* 테스트 데이터를 읽습니다. 
* 테스트 데이터에 대한 예측을 실행합니다. 
* mse 등을 포함하는 분류보고서를 작성합니다. 




In [9]:
# !pygmentize src/xgboost_starter_script.py

# 3. 모델 평가 스텝 개발 및 실행
---



## (1) 로컬에서 스크립트 실행


### 로컬 환경 구성
- 도커 컨테이너의 환경을 가상으로 구성하기 위해 아래 폴더 생성
    - `opt/ml/processing`
    - `opt/ml/processing/evaluation`
    - `opt/ml/processing/model`
    

- 모델 아티펙트 다운로드 하여 로컬에 저장
- 테스트 데이터 세트 다운로드 하여 로컬 저장

In [15]:
import os
base_dir = 'opt/ml/processing'
os.makedirs(base_dir, exist_ok=True)

output_evaluation_dir = 'opt/ml/processing/evaluation'
os.makedirs(output_evaluation_dir, exist_ok=True)

# 훈련 아티펙트를 다운로드 하여 로컬 저장
base_model_dir = 'opt/ml/processing/model'
base_model_path = f"{base_model_dir}/model.tar.gz"
os.makedirs(base_model_dir, exist_ok=True)

print("Download model artifact: ")
! aws s3 cp  {train_model_artifact} {base_model_dir}

# 테스트 데이터 세트의 파일 경로 기술
base_test_path = f"{test_preproc_data_uri}"
print("Test Data Location: \n", base_test_path)


Download model artifact: 
download: s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/training_jobs/pipelines-fi8k402mf07h-FraudScratchTrain-fnnMKXDW3b/output/model.tar.gz to opt/ml/processing/model/model.tar.gz
Test Data Location: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/preporc/test.csv


### 로컬에서 실행 위한 필수 정보 확인

In [18]:
print("base_dir: \n", base_dir)
print("base_model_path: \n", base_model_path)
print("base_test_path: \n", base_test_path)
print("output_evaluation_dir: \n", output_evaluation_dir)

base_dir: 
 opt/ml/processing
base_model_path: 
 opt/ml/processing/model/model.tar.gz
base_test_path: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/preporc/test.csv
output_evaluation_dir: 
 opt/ml/processing/evaluation


### 로컬에서 스크립트 실행

In [19]:
%%sh -s "$base_dir" "$base_model_path" "$base_test_path" "$output_evaluation_dir"
python src/evaluation.py \
--base_dir $1 \
--model_path $2 \
--test_path $3 \
--output_evaluation_dir $4


#############################################
args.model_path: opt/ml/processing/model/model.tar.gz
args.test_path: s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step-phase01/preporc/test.csv
args.output_evaluation_dir: opt/ml/processing/evaluation
****** All folder and files under opt/ml/processing ****** 
('opt/ml/processing', ['evaluation', 'input', 'output', 'model'], [])
('opt/ml/processing/evaluation', ['.ipynb_checkpoints'], ['evaluation.json'])
('opt/ml/processing/evaluation/.ipynb_checkpoints', [], [])
('opt/ml/processing/input', ['.ipynb_checkpoints'], [])
('opt/ml/processing/input/.ipynb_checkpoints', [], [])
('opt/ml/processing/output', ['train', 'test'], [])
('opt/ml/processing/output/train', [], [])
('opt/ml/processing/output/test', ['.ipynb_checkpoints'], [])
('opt/ml/processing/output/test/.ipynb_checkpoints', [], [])
('opt/ml/processing/model', ['.ipynb_checkpoints'], ['model.tar.gz'])
('opt/ml/processing/model/.ipynb_checkpoints', [], [])
*****

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## (2) 로컬 다커 에서 모델 평가


### ScriptProcessor 의 기본 도커 컨테이너 지정
ScriptProcessor 의 기본 도커 컨테이너로 Scikit-learn를 기본 이미지를 사용함. 
- 사용자가 정의한 도커 컨테이너도 사용할 수 있습니다.

In [20]:
# scikit learn 다커 이미지
image_uri = '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'


### ScriptProcessor 정의 및 실행

In [21]:
from sagemaker.processing import ScriptProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput


processing_instance_type = 'local'
# processing_instance_type = 'ml.m4.2xlarge'
eval_script_processor = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-fraud-scratch-eval",
    role=role,
)

eval_script_processor.run(
                        inputs=[
                            ProcessingInput(
#                                source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
                                source= train_model_artifact,  # model_artifcat_path,
                                destination="/opt/ml/processing/model"
                            ),
                            ProcessingInput(
#                                 source=step_process.properties.ProcessingOutputConfig.Outputs[
#                                     "test"
#                                 ].S3Output.S3Uri,
                                source = test_preproc_dir_artifact, # prep_test_output,
                                destination="/opt/ml/processing/test"
                            )
                        ],
                        outputs=[
                            #ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
                            ProcessingOutput(source="/opt/ml/processing/evaluation"),                            
                        ],
                        code="src/evaluation.py",
)


Job Name:  script-fraud-scratch-eval-2021-07-29-06-31-00-962
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/training_jobs/pipelines-fi8k402mf07h-FraudScratchTrain-fnnMKXDW3b/output/model.tar.gz', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-07-29-06-17-04-631/output/test', 'LocalPath': '/opt/ml/processing/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-2-057716757052/script-fraud-scratch-eval-2021-07-29-06-31-00-962/input/code/evaluation.py', 'LocalPath

## (3) 모델 빌딩 파이프라인에서  실행 
---



### 모델 빌딩 파이프라인 변수 생성



In [22]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

### 학습모델을 평가하기 위한 모델 평가단계 정의 



In [23]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-fraud2scratch-eval",
    role=role,
)

### Property 파일 정의

- step_eval 이 실행이 되면 `evaluation.json` 이 S3에 저장이 됩니다.
    - evaluation.json 은 아래의 PropertyFile 로서 정의 됩니다.
    - step_eval 에서 `property_files=[<property_file_instance>]` 를 추가 합니다.

```
<property_file_instance> = PropertyFile(
    name="<property_file_name>",
    output_name="<processingoutput_output_name>",
    path="<path_to_json_file>"
)
```


- 조건 단계에서 사용하는 ConditionLessThanOrEqualTo 에서 evaluation.json 을 로딩하여 내용을 확인

```
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=<property_file_instance>,
        json_path="test_metrics.roc.value",
    ),
    right=6.0
)
```

#### 참고
- 참고 자료: [Property Files and JsonGet](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/build-and-manage-propertyfile.html)

In [24]:
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import ProcessingStep

from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)


step_eval = ProcessingStep(
    name="FraudEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source= train_model_artifact,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source= test_preproc_dir_artifact,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="src/evaluation.py",
    property_files=[evaluation_report],
)

### 조건  단계 정의
- step_eval 의 결과가 조건 단계로 연결되기에 아래 추가하여 진행 합니다.

In [25]:
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)


cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value",
    ),
    right=6.0
)

step_cond = ConditionStep(
    name="AbaloneMSECond",
    conditions=[cond_lte],
#    if_steps=[step_register, step_create_model, step_transform],
    if_steps=[],    
    else_steps=[], 
)

### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [26]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
    ],
   steps=[step_eval, step_cond],
)

### 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [27]:
import json


definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 파이프라인 서비스에 제출합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [28]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase01',
 'ResponseMetadata': {'RequestId': '33b1dbeb-be79-4f67-9d74-ab95b793c58b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '33b1dbeb-be79-4f67-9d74-ab95b793c58b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '112',
   'date': 'Thu, 29 Jul 2021 06:31:18 GMT'},
  'RetryAttempts': 0}}

In [29]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [30]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase01',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step-phase01/execution/tmwx82bczbgy',
 'PipelineExecutionDisplayName': 'execution-1627540279084',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 7, 29, 6, 31, 19, 24000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 7, 29, 6, 31, 19, 24000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': '7fbcf1ce-6481-437c-8963-aaf911708c82',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7fbcf1ce-6481-437c-8963-aaf911708c82',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '453',
   'date': 'Thu, 29 Jul 2021 06:31:18 GMT'},
  'RetryAttempts': 0}}

In [ ]:

execution.wait()

In [ ]:
execution.list_steps()